# Personalize with temporal evaluation on hold-out set

The notebook largely follows the basic notebook, with the additional tweak to hold-out 10% of "future" data for every user. Then, we set up an inference endpoint to bring recommendation and evaluate externally on the held-out data.

The other minor difference is that we predict views rather than ratings. We find it more interesting to demonstrate recommendation of unpopular (yet highly personalized) movies than to recommend popular movies that everyone would enjoy (without personalization needs).

In [1]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time
import tempfile, urllib, zipfile, subprocess

In [3]:
os.environ['AWS_DEFAULT_REGION']="us-west-2"

In [4]:
suffix = str(np.random.uniform())[4:9]

In [5]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [6]:
!aws s3 mb s3://{bucket}

make_bucket: demo-temporal-holdout-27346


In [7]:
personalize = boto3.client('personalize', endpoint_url='https://personalize.us-west-2.amazonaws.com')
personalize_runtime = boto3.client('personalize-runtime', endpoint_url='https://personalize-runtime.us-west-2.amazonaws.com')

# Download and process benchmark data

In [8]:
with tempfile.TemporaryDirectory() as tmpdir:
    urllib.request.urlretrieve(
        'http://files.grouplens.org/datasets/movielens/ml-1m.zip',
        tmpdir + '/ml-1m.zip')
    zipfile.ZipFile(tmpdir + '/ml-1m.zip').extractall(tmpdir)
    print(subprocess.check_output(['ls', tmpdir+'/ml-1m']).decode('utf-8'))
    
    data = pd.read_csv(
        tmpdir + '/ml-1m/ratings.dat',
        sep='::',
        names=['USER_ID','ITEM_ID','EVENT_VALUE', 'TIMESTAMP'])

movies.dat
ratings.dat
README
users.dat



/home/yifeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()


In [9]:
pd.set_option('display.max_rows', 5)

In [10]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

unique users 6040; unique items 3706


,USER_ID,ITEM_ID,TIMESTAMP
0,1,1193,978300760
1,1,661,978302109
...,...,...,...
1000207,6040,1096,956715648
1000208,6040,1097,956715569


### Extract last 10% of interactions per user as hold-out tests

In [11]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [12]:
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

unique users 6040; unique items 3684


,USER_ID,ITEM_ID,TIMESTAMP
0,1,1193,978300760
1,1,661,978302109
...,...,...,...
1000207,6040,1096,956715648
1000208,6040,1097,956715569


### Upload data

In [13]:
data.to_csv(filename, index=False)
!head {filename}
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

USER_ID,ITEM_ID,TIMESTAMP
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,1197,978302268
1,1287,978302039
1,2804,978300719
1,594,978302268
1,919,978301368


# Create Schema

In [14]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-west-2:261294318658:schema/DEMO-temporal-schema-27346",
  "ResponseMetadata": {
    "RequestId": "598d595a-189c-4573-b33a-857e41644f85",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 02:53:39 GMT",
      "x-amzn-requestid": "598d595a-189c-4573-b33a-857e41644f85",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create and Wait for Dataset Group

In [15]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-west-2:261294318658:dataset-group/DEMO-temporal-dataset-group-27346",
  "ResponseMetadata": {
    "RequestId": "2ef7ee25-7237-406d-b04a-bf06add6cb51",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 02:53:40 GMT",
      "x-amzn-requestid": "2ef7ee25-7237-406d-b04a-bf06add6cb51",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [16]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


In [17]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-west-2:261294318658:dataset/DEMO-temporal-dataset-group-27346/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "90fe3b25-07ee-440f-9073-ed0ec017d1ae",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 02:54:20 GMT",
      "x-amzn-requestid": "90fe3b25-07ee-440f-9073-ed0ec017d1ae",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Prepare, Create, and Wait for Dataset Import Job

In [18]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### Create S3 Read-Only Access Role

In [19]:
from botocore.exceptions import ClientError

In [20]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
print(role_arn)

arn:aws:iam::261294318658:role/PersonalizeS3Role-27346


In [21]:
time.sleep(20) # wait for iam role creation

In [22]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:261294318658:dataset-import-job/DEMO-temporal-dataset-import-job-27346",
  "ResponseMetadata": {
    "RequestId": "52469899-6a8c-48df-8190-65f702ad0f8f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 02:54:41 GMT",
      "x-amzn-requestid": "52469899-6a8c-48df-8190-65f702ad0f8f",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

In [23]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


# Create Solution

In [24]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims


There are many recipes for different scenarios. In this example, we only have interactions data, so we will choose one from the basic recipes.

| Feasible? | Recipe | Description 
|-------- | -------- |:------------
| Y | aws-popularity-count | Calculates popularity of items based on count of events against that item in user-item interactions dataset.
| Y | aws-hrnn | Predicts items a user will interact with. A hierarchical recurrent neural network which can model the temporal order of user-item interactions.
| N - requires meta data | aws-hrnn-metadata | Predicts items a user will interact with. HRNN with additional features derived from contextual (user-item interaction metadata), user medata (user dataset) and item metadata (item dataset)
| N - for bandits and requires meta data | aws-hrnn-coldstart | Predicts items a user will interact with. HRNN-metadata with with personalized exploration of new items.
| N - for item-based queries | aws-sims | Computes items similar to a given item based on co-occurrence of item in same user history in user-item interaction dataset
| N - for reranking a short list | aws-personalized-ranking | Reranks a list of items for a user. Trains on user-item interactions dataset. 


We (or autoML) can run all of these basic recipes and choose the best-performing model from internal metrics. We recommend comparisons, especially with popularity-baseline, to see the lifts in metrics via personalization. However, in this demo, we will pick one recipe - aws-hrnn, to focus on external evaluations.

In [25]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

In [26]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-west-2:261294318658:solution/DEMO-temporal-solution-27346",
  "ResponseMetadata": {
    "RequestId": "f1a35677-715d-4fce-974d-2db24ddbca31",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 03:11:43 GMT",
      "x-amzn-requestid": "f1a35677-715d-4fce-974d-2db24ddbca31",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:261294318658:solution/DEMO-temporal-solution-27346/3e4e98a3",
  "ResponseMetadata": {
    "RequestId": "24c86670-9b1a-4aad-8b3b-c15989f51749",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 03:11:43 GMT",
      "x-amzn-requestid": "24c86670-9b1a-4aad-8b3b-c15989f51749",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Solution Version to Have ACTIVE Status

In [28]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

### Get Metrics of Solution

In [29]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:261294318658:solution/DEMO-temporal-solution-27346/3e4e98a3",
  "metrics": {
    "coverage": 0.5685,
    "mean_reciprocal_rank_at_25": 0.0799,
    "normalized_discounted_cumulative_gain_at_10": 0.1142,
    "normalized_discounted_cumulative_gain_at_25": 0.1509,
    "normalized_discounted_cumulative_gain_at_5": 0.0841,
    "precision_at_10": 0.0202,
    "precision_at_25": 0.014,
    "precision_at_5": 0.0227
  },
  "ResponseMetadata": {
    "RequestId": "1a8667cd-c2d1-479c-b85c-ee2abd3a633c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 03:53:46 GMT",
      "x-amzn-requestid": "1a8667cd-c2d1-479c-b85c-ee2abd3a633c",
      "content-length": "411",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create and Wait for Campaign

In [30]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-west-2:261294318658:campaign/DEMO-temporal-campaign-27346",
  "ResponseMetadata": {
    "RequestId": "00c1808d-cb81-40c3-9573-31e206aeea67",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 11 Oct 2019 03:53:45 GMT",
      "x-amzn-requestid": "00c1808d-cb81-40c3-9573-31e206aeea67",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Campaign to Have ACTIVE Status

In [31]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


# Evaluate using external metrics

An explanation of the evaluation metrics are provided at https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html

For example, suppose we recommend four items and two of them are relevant, $r=[0,1,0,1]$. In this case, the metrics are:

|Name	|Example	|Explanation
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|Total relevant items divided by total recommended items.
|Mean reciprocal ranks (MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|Considers positional effects by computing the mean of the inverse positions of all relevant items.
|Normalized discounted cumulative gains (NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|Considers positional effects by applying inverse logarithmic weights based on the positions of relevant items, normalized by the largest possible scores from ideal recommendations.
|Average precision (AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Average precision@K where K is the position of every relevant item.

These metrics are different from the internal metrics in two aspects:
* They are evaluated at different times, which may imply different click rates. We recommend to always keep the evaluations in the same time periods to avoid temporal drifts.
* The example external evaluations may hold out and consider multiple items as ground truth, while the internal evaluations only hold out the last item in each user-history as the ground truth. There is no absolute preference as to how many items should be held out; we recommend designing the evaluation methods that are similar to the actual use case.

In [32]:
from tqdm import tqdm_notebook
import numpy as np
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [33]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [34]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

mean_reciprocal_rank 0.27763984016842114
precision_at_5 0.12225165562913907
precision_at_10 0.10912251655629139
precision_at_25 0.08934437086092717
normalized_discounted_cumulative_gain_at_5 0.18794320731200617
normalized_discounted_cumulative_gain_at_10 0.24846691673991717
normalized_discounted_cumulative_gain_at_25 0.36137096907190547


### Optional: slightly better results after deduplicating previous purchase histories

In [35]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [36]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

mean_reciprocal_rank 0.34547569737777906
precision_at_5 0.16907284768211922
precision_at_10 0.1479635761589404
precision_at_25 0.08934437086092717
normalized_discounted_cumulative_gain_at_5 0.2638009850020756
normalized_discounted_cumulative_gain_at_10 0.3420073299809215
normalized_discounted_cumulative_gain_at_25 0.4211565003372475


### Try comparing with popularity baseline as a dummy recommender

In [37]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [38]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [39]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

mean_reciprocal_rank 0.06435930491569826
precision_at_5 0.01980132450331126
precision_at_10 0.020066225165562918
precision_at_25 0.021735099337748344
normalized_discounted_cumulative_gain_at_5 0.043792523162000045
normalized_discounted_cumulative_gain_at_10 0.06552215768085591
normalized_discounted_cumulative_gain_at_25 0.12216764300043037


### Popularity baseline deduplicating user histories

In [40]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [41]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

mean_reciprocal_rank 0.1476091094743245
precision_at_5 0.06321192052980133
precision_at_10 0.055993377483443706
precision_at_25 0.04464238410596026
normalized_discounted_cumulative_gain_at_5 0.10508158681030984
normalized_discounted_cumulative_gain_at_10 0.14086321546662076
normalized_discounted_cumulative_gain_at_25 0.21005097965654965


# clean up

In [42]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

{'ResponseMetadata': {'RequestId': 'e849eb0e-44a2-445c-a8a9-872ea8a68ebb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 11 Oct 2019 05:26:25 GMT',
   'x-amzn-requestid': 'e849eb0e-44a2-445c-a8a9-872ea8a68ebb',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

# execute with caution!

In [43]:
!aws s3 rm s3://{bucket} --recursive

delete: s3://demo-temporal-holdout-27346/DEMO-temporal-holdout.csv
